In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
posters = pd.read_csv('movie_poster.csv')

In [3]:
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))','')
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies

C:\Users\Oindrila\AppData\Local\Temp/ipykernel_1980/3418239941.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))','')


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy
9739,193585,Flint,Drama
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation


In [4]:
movies = movies.drop('genres',1)

C:\Users\Oindrila\AppData\Local\Temp/ipykernel_1980/3798460745.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies = movies.drop('genres',1)


In [5]:
movies.head()

,movieId,title
0,1,Toy Story
1,2,Jumanji
2,3,Grumpier Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
ratings = ratings.drop('timestamp',1)

C:\Users\Oindrila\AppData\Local\Temp/ipykernel_1980/2271291715.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings = ratings.drop('timestamp',1)


In [8]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [9]:
movies.head()

,movieId,title
0,1,Toy Story
1,2,Jumanji
2,3,Grumpier Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


# Creating the user profile

This function is for making the user profile to which the movies will be recommended to

In [10]:
user = [
    {'title':'Toy Story','rating':4},
    {'title':'Jumanji','rating':3.5},
    {'title':'Casino','rating':2},
    {'title':'Othello','rating':2.5},
    {'title':'Babe','rating':5}
]

input = pd.DataFrame(user)
input

,title,rating
0,Toy Story,4.0
1,Jumanji,3.5
2,Casino,2.0
3,Othello,2.5
4,Babe,5.0


In [11]:
inputId = movies[movies['title'].isin(input['title'].tolist())]

input = pd.merge(inputId, input)

input = input.merge(posters,on='title')

In [12]:
input

,movieId,title,rating,Id,poster
0,1,Toy Story,4.0,1,https://images-na.ssl-images-amazon.com/images...
1,2,Jumanji,3.5,55,https://www.bing.com/th?id=OIP.WfVGQ9Z4JO-U7h6...
2,16,Casino,2.0,57,https://www.bing.com/th?id=OIP.oaOuY8fSh7TELiR...
3,26,Othello,2.5,56,https://www.bing.com/th?id=OIP.MtDUrfDLwruAg1O...
4,34,Babe,5.0,7,https://images-na.ssl-images-amazon.com/images...


In [52]:
similarUser = ratings[ratings['movieId'].isin(input['movieId'].tolist())]
similarUser.head()

,userId,movieId,rating
0,1,1,4.0
516,5,1,4.0
518,5,34,4.0
560,6,2,4.0
571,6,16,4.0


In [53]:
usersGrp = similarUser.groupby(['userId'])

In [54]:
usersGrp.get_group(610)

,userId,movieId,rating
99534,610,1,5.0
99536,610,16,4.5


In [55]:
#Sorts the user subsets according to the highest priority of similarity to the input user
usersGrp = sorted(usersGrp, key=lambda x: len(x[1]), reverse=True)

In [56]:
#the userId who has watched the most number of common movies with the input user
usersGrp[0]

(68,
        userId  movieId  rating
 10360      68        1     2.5
 10361      68        2     2.5
 10368      68       16     3.5
 10373      68       26     3.0
 10375      68       34     1.0)

In [57]:
#Dataframe of the topmost priority user
usersGrp[0][1]

,userId,movieId,rating
10360,68,1,2.5
10361,68,2,2.5
10368,68,16,3.5
10373,68,26,3.0
10375,68,34,1.0


# Pearson Correlation for finding similarity

This function is used to find the similarity between the input user with a subset of common users

In [58]:
usersGrp = usersGrp[0:200]

In [59]:
pearsonCoDict = {}

for name, group in usersGrp:
    
    group = group.sort_values(by='movieId')
    input = input.sort_values(by='movieId')
    
    n = len(group)
    
    temp = input[input['movieId'].isin(group['movieId'].tolist())]
    
    rateList = temp['rating'].tolist()
    
    grpList = group['rating'].tolist()
    
    #scipy.stats.pearsonr(rateList, grpList)[0]
    
    Sxx = sum([i**2 for i in rateList]) - pow(sum(rateList),2)/float(n)
    Syy = sum([i**2 for i in grpList]) - pow(sum(grpList),2)/float(n)
    Sxy = sum(i*j for i, j in zip(rateList, grpList)) - sum(rateList)*sum(grpList)/float(n)
    
    if Sxx != 0 and Syy != 0:
        pearsonCoDict[name] = Sxy/np.sqrt(Sxx*Syy)
        
    else:
        pearsonCoDict[name] = 0

In [60]:
pearsonCoDict.items()

dict_items([(68, -0.9515190335342454), (474, 0.42246305606312334), (6, 0), (18, -0.9173460685716212), (103, -0.8666666666666667), (240, 0.8666666666666667), (274, -0.48989794855663565), (330, -0.06172133998483677), (357, 0.4098780306383839), (373, -0.11547005383792514), (380, 0.5222329678670935), (414, 0.8703882797784892), (470, 0.8320502943378437), (480, -0.11547005383792514), (483, 0), (599, 0.0), (608, -0.4359546962416407), (19, 0.7559289460184538), (27, 0.6546536707079778), (40, 0.9176629354822458), (64, -0.6546536707079773), (82, -0.9819805060619666), (91, -0.576556660197054), (93, 0.18898223650461524), (112, -0.7205766921228919), (117, -0.8029550685469663), (140, 0.6546536707079778), (144, -0.1889822365046185), (160, 0.9449111825230734), (169, -0.18898223650462054), (177, 0.8461538461538475), (182, -0.6933752452815394), (191, 0), (201, 0.9449111825230636), (217, 0.755928946018457), (226, -0.8386278693775367), (288, 0.6185895741317425), (298, -0.7751332793988399), (305, -0.9819805

In [61]:
pearson = pd.DataFrame.from_dict(pearsonCoDict, orient='index')
pearson.head()

,0
68,-0.951519
474,0.422463
6,0.000000
18,-0.917346
103,-0.866667


In [62]:
pearson.columns = ['similarity']
pearson['userId'] = pearson.index
pearson.index = range(len(pearson))
pearson.head()

,similarity,userId
0,-0.951519,68
1,0.422463,474
2,0.000000,6
3,-0.917346,18
4,-0.866667,103


# Top 50 similar users to input user

In [63]:
topUsers = pearson.sort_values(by='similarity', ascending = False)[0:50]
topUsers.head()

,similarity,userId
143,1.0,610
107,1.0,282
126,1.0,482
127,1.0,484
122,1.0,412


In [64]:
topUsersRating = topUsers.merge(ratings, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarity,userId,movieId,rating
0,1.0,610,1,5.0
1,1.0,610,6,5.0
2,1.0,610,16,4.5
3,1.0,610,32,4.5
4,1.0,610,47,5.0


In [65]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarity']*topUsersRating['rating']
topUsersRating.head()

,similarity,userId,movieId,rating,weightedRating
0,1.0,610,1,5.0,5.0
1,1.0,610,6,5.0,5.0
2,1.0,610,16,4.5,4.5
3,1.0,610,32,4.5,4.5
4,1.0,610,47,5.0,5.0


In [66]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarity','weightedRating']]
tempTopUsersRating.columns = ['sum_similarity','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarity,sum_weightedRating
movieId,,
1,32.421653,135.673941
2,27.475583,93.764732
3,6.699553,21.942445
5,5.499420,13.375588
6,15.014486,54.964179


In [67]:
#Empty dataframe to store the recommendation score
recommendation = pd.DataFrame()
#Weighted average for calculating the weighted average recommendation score
recommendation['recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarity']
recommendation['movieId'] = tempTopUsersRating.index
recommendation.head()

,recommendation score,movieId
movieId,,
1,4.184671,1
2,3.412657,2
3,3.275210,3
5,2.432182,5
6,3.660743,6


In [68]:
recommendation = recommendation.sort_values(by='recommendation score', ascending=False)
recommendation.head()

,recommendation score,movieId
movieId,,
290,5.0,290
115727,5.0,115727
92420,5.0,92420
5867,5.0,5867
138632,5.0,138632


In [69]:
recommended_df = movies.loc[movies['movieId'].isin(recommendation.head(10)['movieId'].tolist())]
recommended_df = recommended_df.merge(posters,on='title')

In [70]:
recommended_df.head()

,movieId,title,Id,poster
0,290,Once Were Warriors,32,https://www.bing.com/th?id=OIP.JBB8RkPHTzXujF2...
1,3404,Titanic,33,https://www.bing.com/th?id=OIP.nZPRRhZtsc7qevm...
2,5867,Thief,34,https://www.bing.com/th?id=OIP.CvEjkZFbIF8jYbF...
3,5909,Visitor Q (Bizita Q),35,https://www.bing.com/th?id=OIP.P11RIWBeb2KPJFL...
4,6791,Babette's Feast (Babettes gæstebud),36,https://th.bing.com/th/id/OIP.vCa4HXNq8Itt9F2V...


# Another approach

In [71]:
ratings = ratings.merge(movies,on='movieId')

In [72]:
ratings.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,5,1,4.0,Toy Story
2,7,1,4.5,Toy Story
3,15,1,2.5,Toy Story
4,17,1,4.5,Toy Story


In [73]:
x = ratings.groupby('userId').count()['rating'] > 200
popular_users = x[x].index

In [74]:
filtered_ratings = ratings[ratings['userId'].isin(popular_users)]

In [75]:
y = filtered_ratings.groupby('title').count()['rating']>=20
popular_movies = y[y].index

In [76]:
final_ratings = filtered_ratings[filtered_ratings['title'].isin(popular_movies)]

In [77]:
final_ratings

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
5,18,1,3.5,Toy Story
6,19,1,4.0,Toy Story
7,21,1,3.5,Toy Story
15,45,1,4.0,Toy Story
...,...,...,...,...
99625,509,130073,4.0,Cinderella
99688,517,144352,0.5,Unforgiven
100363,599,103027,3.0,Much Ado About Nothing
100585,606,5990,3.5,Pinocchio


In [78]:
pt = final_ratings.pivot_table(index='title',columns='userId',values='rating')

In [79]:
pt

userId,1,4,6,18,19,20,21,28,41,42,...,594,596,597,599,600,603,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.5,NaN,...,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,3.5
10 Things I Hate About You,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,3.0,3.0,3.0,5.0,NaN,NaN,NaN
101 Dalmatians,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,4.0,3.0,NaN,NaN,NaN
101 Dalmatians (One Hundred and One Dalmatians),NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
12 Angry Men,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zodiac,NaN,NaN,NaN,4.5,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5.0
Zombieland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.5
Zoolander,NaN,NaN,NaN,3.0,NaN,3.5,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,3.0,4.0


In [80]:
pt.fillna(0,inplace=True)

In [81]:
pt

userId,1,4,6,18,19,20,21,28,41,42,...,594,596,597,599,600,603,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.5,0.0,...,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,3.5
10 Things I Hate About You,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,3.0,3.0,3.0,5.0,0.0,0.0,0.0
101 Dalmatians,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,0.0
101 Dalmatians (One Hundred and One Dalmatians),0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zodiac,0.0,0.0,0.0,4.5,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0
Zombieland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.5
Zoolander,0.0,0.0,0.0,3.0,0.0,3.5,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,3.0,4.0


In [82]:
from sklearn.metrics.pairwise import cosine_similarity

In [83]:
similarity_scores = cosine_similarity(pt)

In [84]:
similarity_scores.shape

(952, 952)

In [85]:
def recommend(title):
    #fetching indices
    index = np.where(pt.index==title)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:6]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = movies[movies['title'] == pt.index[i[0]]]
        temp_df = temp_df.merge(posters,on='title')
        item.extend(list(temp_df.drop_duplicates('title')['title'].values))
        item.extend(list(temp_df.drop_duplicates('movieId')['movieId'].values))
        item.extend(list(temp_df.drop_duplicates('poster')['poster'].values))
        
        data.append(item)
        
    return data

In [201]:
recommend('Toy Story')

[['Jurassic Park',
  480,
  'https://www.bing.com/th?id=OIP.DjNHqQrUWPcviBMFx5svuAHaLH&w=150&h=225&c=8&rs=1&qlt=90&o=6&dpr=1.25&pid=3.1&rm=2'],
 ['Star Wars: Episode IV - A New Hope',
  260,
  'https://www.bing.com/th?id=OIP.lqI1xEzwm5y0pfjwyXDWTAHaKj&w=150&h=214&c=8&rs=1&qlt=90&o=6&dpr=1.25&pid=3.1&rm=2'],
 ['Shrek',
  4306,
  'https://www.bing.com/th?id=OIP.9Hj-aAhsxdauWdKVwrMorwHaLH&w=150&h=225&c=8&rs=1&qlt=90&o=6&dpr=1.25&pid=3.1&rm=2'],
 ['Star Wars: Episode V - The Empire Strikes Back',
  1196,
  'https://www.bing.com/th?id=OIP.ZWfQ9SECZNvQXI75loSbXQHaKj&w=150&h=214&c=8&rs=1&qlt=90&o=6&dpr=1.25&pid=3.1&rm=2'],
 ['Star Wars: Episode VI - Return of the Jedi',
  1210,
  'https://www.bing.com/th?id=OIP.f0L8dG4zRa6v9VhnP29nMQHaLH&w=150&h=225&c=8&rs=1&qlt=90&o=6&dpr=1.25&pid=3.1&rm=2']]

In [202]:
import pickle
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(movies,open('movies.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))

In [203]:
pickle.dump(recommended_df,open('recommended_df.pkl','wb'))
pickle.dump(posters,open('posters.pkl','wb'))